In [1]:
cd

/Users/hanjialu


In [4]:
import tensorflow as tf
import scipy.io
import numpy as np
# import matplotlib.pyplot as plt
import time

# % matplotlib inline

In [5]:
# Remember change to your own path

path = "Documents/p5/data/"
train = scipy.io.loadmat(path + 'train_32x32.mat')
test = scipy.io.loadmat(path + 'test_32x32.mat')

x_train = train['X']
y_train = train['y']
x_test = test['X']
y_test = test['y']

In [6]:
# Reshape the training data and testing data

def reformat(data):
    lenth = data['X'].shape[3]
    X_train = []
    y_train = np.zeros((lenth,10))
    
    for i in range(lenth):
        X_train.append(data['X'][:,:,:,i])
        y_train[i][data['y'][i]%10] = 1
    
    return np.array(X_train).astype('float32') ,np.array(y_train).astype('float32')

In [7]:
X_train,y_train = reformat(train)
X_test,y_test = reformat(test)

In [8]:
"""
Various Hyperparameters required for training the CNN.
"""
image_size = 32
width = 32
height = 32
channels = 3

n_labels = 10
patch = 5
depth = 16
hidden = 128

In [9]:
"""
Constructing the placeholders and variables in the TensorFlow Graph
"""
#Training Dataset
tf_train_dataset = tf.placeholder(tf.float32, shape=(None, width, height, channels))
#Training Labels
tf_train_labels = tf.placeholder(tf.float32, shape=(None, n_labels))
#Testing Dataset
tf_test_dataset = tf.constant(X_test)

#   Layer 1: (5, 5, 3, 16)
layer1_weights = tf.Variable(tf.truncated_normal([patch, patch, channels, depth], stddev=0.1))
layer1_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

#   Layer 2: (5, 5, 16, 16)
layer2_weights = tf.Variable(tf.truncated_normal([patch, patch, depth, depth], stddev=0.1))
layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

#   Layer 3: (1024, 128)
layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, hidden], stddev=0.1))
layer3_biases = tf.Variable(tf.constant(1.0, shape=[hidden]))

#   Layer 4: (128, 10)
layer4_weights = tf.Variable(tf.truncated_normal([hidden, n_labels], stddev=0.1))
layer4_biases = tf.Variable(tf.constant(1.0, shape=[n_labels]))

dropout = tf.placeholder(tf.float32)

In [10]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [11]:
def model(data):
    #   Convolution 1 and RELU
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden1 = tf.nn.relu(conv1 + layer1_biases)
    #   Max Pool
    hidden2 = tf.nn.max_pool(hidden1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    #   Convolution 2 and RELU
    conv2 = tf.nn.conv2d(hidden2, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden3 = tf.nn.relu(conv2 + layer2_biases)
    #   Max Pool
    hidden4 = tf.nn.max_pool(hidden3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    shape = hidden4.get_shape().as_list()

    reshape = tf.reshape(hidden4, [-1, shape[1] * shape[2] * shape[3]])
    hidden5 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    #   Dropout
    dropout_layer = tf.nn.dropout(hidden5, 0.93)

    return tf.matmul(dropout_layer, layer4_weights) + layer4_biases

logits = model(tf_train_dataset)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

train_prediction = tf.nn.softmax(logits)
test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [12]:
# Parameters

drop_out = 0.5

batch = 16
learning_rate = 0.001
num_steps = 60

In [17]:
# transfer one hot expression to number

def oneHotToNumber(lists):
    nums = [i for li in lists for i in range(10) if li[i] == 1]
    return nums

In [14]:
saver = tf.train.Saver()

In [43]:
# from scipy.misc import imread, imsave, imresize
# # get the picture for local folder
# path = "Documents/P5/"
# img = imread(path+'4.png')[:,:,:3]
# img = imresize(img, (32, 32))

# Predict
path = "Documents/p5/Model/Model2/"

sess=tf.Session()
new_saver = tf.train.import_meta_graph(path+'my-model.meta')
new_saver.restore(sess, path+'my-model')


In [63]:
from scipy.misc import imread, imsave, imresize
import time
# get the picture for local folder
path = "Desktop/"
img = imread(path+'2.png')[:,:,:3]/128.0-1

t0 = time.time()
img = imresize(img, (32, 32))

feed_dict = {tf_train_dataset : [img]}
classification = sess.run(train_prediction, feed_dict)
print time.time()-t0
print oneHotToNumber(classification)


0.00597405433655
[2]
